<a href="https://colab.research.google.com/github/sheon-j/webcrawling-study/blob/main/kid_donga_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web Crawling 어린이 동아

In [188]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import json

In [189]:
def crawl_href_list(page):
    url = f"http://kids.donga.com/?page_no={page}&ptype=article&psub=news&gbn=01"
    req = requests.get(url)
    html = req.text
    soup = BeautifulSoup(html, "html.parser")
    href_list = [a.get("href") for a in soup.select("dt > a")]
    return href_list

def to_json_data(href):
    pattern = re.compile("[\xa0\u200b\n\r\t]")
    json_data = {}

    # sub_soup
    sub_req = requests.get(href)
    sub_html = sub_req.text
    sub_soup = BeautifulSoup(sub_html, "html.parser")

    # news_url
    news_url = href
    json_data["news_url"] = news_url

    # news_title
    news_title = sub_soup.select_one("li.title").text
    news_title = re.sub(pattern, "", news_title).strip()
    json_data["news_title"] = news_title

    # news_subtitle
    news_subtitle = ""
    json_data["news_subtitle"] = news_subtitle

    # news_writer
    news_writer = sub_soup.select_one("li.writer").text
    news_writer = re.sub(pattern, "", news_writer.split(" ")[0]).strip()
    json_data["news_writer"] = news_writer

    # news_date
    news_date = sub_soup.select_one("li.upload_date").text.strip()
    json_data["news_date"] = news_date

    # news_article
    news_article = sub_soup.select("div.at_content > p")
    news_article = [
        p.text.strip() for p in news_article 
        if not p.img 
        if not "위 기사의 법적인 책임과 권한은" in p.text 
        if not "< 저작권자 ⓒ 어린이동아" in p.text 
        if not ("▶어린이동아" and "기자") in p.text
    ]
    news_article = " ".join(news_article)
    news_article = re.sub(pattern, "", news_article).strip()
    json_data["news_article"] = news_article

    # news_img
    news_img = sub_soup.select_one("div.at_content img").get("src")
    news_img = "" if len(news_img) >= 200 else news_img
    json_data["news_img"] = news_img

    # news_source
    news_source = "어린이동아"
    json_data["news_source"] = news_source

    return json_data

In [195]:
page = 1
data = []
date_pivot = "2022-02-20"
date_pivot = datetime.strptime(date_pivot, "%Y-%m-%d")

while True:
    href_list = crawl_href_list(page)
    for href in href_list:
        json_data = to_json_data(href)
        date_obj = json_data["news_date"].split(" ")[0]
        date_obj = datetime.strptime(date_obj, "%Y-%m-%d")
        if date_pivot > date_obj:
            break
        data.append(json_data)
    
    print(page, "done!")
    if date_pivot > date_obj:
            break
    page += 1

print("total data: ", len(data))

1 done!
2 done!
3 done!
total data:  27


In [ ]:
data

In [183]:
# json 파일로 저장
with open("kids_donga.json", "w", encoding="utf-8") as json_file:
    json.dump(data, json_file,  ensure_ascii=False)